In [ ]:
import json
import time
import numpy as np
import tensorflow as tf

In [ ]:
from statistics import median
from nltk.tokenize import word_tokenize
from sklearn.metrics import f1_score, confusion_matrix
from sklearn.utils import shuffle

In [ ]:
def convert_json_to_review_and_rating(json_text):
    review_dict = json.loads(json_text)    
    return review_dict['reviewText'], review_dict['overall']

In [ ]:
def get_reviews_and_ratings(reviews_filepath):
    review_texts = list()
    ratings = list()
    with open(reviews_filepath) as reviews_file:
        for line in reviews_file:
            review_text, rating = convert_json_to_review_and_rating(line)
            review_texts.append(review_text)
            ratings.append(int(rating))
            
    return review_texts, ratings

In [ ]:
def texts_to_indexed_word_sequences(review_texts):
    word_indices = dict()
    indexed_sequences = list()
    word_index = 1
    
    for review_text in review_texts:
        tokens = word_tokenize(review_text)
        indexed_sequence = list()
        for token in tokens:
            if token not in word_indices:
                word_indices[token] = word_index
                indexed_sequence.append(word_index)
                word_index += 1
            else:
                indexed_sequence.append(word_indices[token])
        indexed_sequences.append(np.asarray(indexed_sequence))
        
    return word_indices, indexed_sequences

In [ ]:
reviews_filepath = "/home/v2john/datasets/amazon/reviews_electronics.json"

In [ ]:
review_texts, ratings = get_reviews_and_ratings(reviews_filepath)
review_texts, ratings = shuffle(review_texts, ratings)
print(len(review_texts), len(ratings))

In [ ]:
word_indices, indexed_sequences = texts_to_indexed_word_sequences(review_texts)

In [ ]:
VOCAB_SIZE = len(word_indices)
print("VOCAB_SIZE: ", VOCAB_SIZE)

EMBEDDING_SIZE = 300
print("EMBEDDING_SIZE: ", EMBEDDING_SIZE)

MAX_SEQUENCE_LENGTH = int(median([len(sequence) for sequence in indexed_sequences]))
print("MAX_SEQUENCE_LENGTH: ", MAX_SEQUENCE_LENGTH)

NUM_CLASSES = len(set(ratings))
print("NUM_CLASSES: ", NUM_CLASSES)

In [ ]:
def pad_indexed_sequences(indexed_sequences, max_sequence_length):
    new_indexed_sequences = list()
    for sequence in indexed_sequences:
        if len(sequence) >= max_sequence_length:
            new_indexed_sequences.append(sequence[:max_sequence_length])
        else:
            shortfall = max_sequence_length - len(sequence)
            new_indexed_sequences.append(
                np.pad(sequence, (0, shortfall), 'constant', 
                       constant_values=(0, 0)))
    return np.asarray(new_indexed_sequences)

def convert_labels_to_logits(ratings, num_classes):
    one_hot_ratings = list()
    for rating in ratings:
        one_hot_rating = np.zeros(num_classes)
        one_hot_rating[rating - 1] = 1
        one_hot_ratings.append(one_hot_rating)
        
    return np.asarray(one_hot_ratings)

def tensorize_sequences_and_labels(indexed_sequences, ratings, max_sequence_length, num_classes):
    return pad_indexed_sequences(indexed_sequences, max_sequence_length), \
        convert_labels_to_logits(ratings, num_classes)

In [ ]:
indexed_sequences, labels = tensorize_sequences_and_labels(
    indexed_sequences, ratings, MAX_SEQUENCE_LENGTH, NUM_CLASSES)

In [ ]:
indexed_sequences.shape, labels.shape

In [ ]:
graph_1 = tf.Graph()
current_epoch = int(time.time())

with graph_1.as_default():

    input_sequence = tf.placeholder(
        tf.int32, [None, MAX_SEQUENCE_LENGTH], name="input_sequence")
    print("input_sequence: ", input_sequence)
    
    input_label = tf.placeholder(
        tf.int32, [None, NUM_CLASSES], name="input_label")

    word_embeddings = tf.get_variable(
        shape=[VOCAB_SIZE + 1, EMBEDDING_SIZE], name="word_embeddings", 
        dtype=tf.float32)
    print("word_embeddings: ", word_embeddings)

    embedded_sequence = tf.nn.embedding_lookup(
        word_embeddings, input_sequence, name="embedded_sequence")
    print("embedded_sequence: ", embedded_sequence)

        
    # Discriminator Network
    conv_1 = tf.layers.conv1d(
        inputs=embedded_sequence, filters=64, kernel_size=3, 
        activation=tf.nn.relu, name="conv_1")
    print("conv_1: ", conv_1)

    bnorm_1 = tf.layers.batch_normalization(conv_1, name="batch_norm_1")
    print("batch_norm_1: ", bnorm_1)

    max_pool_1 = tf.layers.max_pooling1d(
        inputs=conv_1, pool_size=2, strides=2, name="max_pool_1")
    print("max_pool_1: ", max_pool_1)

    conv_2 = tf.layers.conv1d(
        inputs=max_pool_1, filters=128, kernel_size=3, 
        activation=tf.nn.relu, name="conv_2")
    print("conv_2: ", conv_2)

    bnorm_2 = tf.layers.batch_normalization(conv_2, name="batch_norm_2")
    print("batch_norm_2: ", bnorm_2)

    max_pool_2 = tf.layers.max_pooling1d(
        inputs=conv_2, pool_size=2, strides=2, name="max_pool_2")
    print("max_pool_2: ", max_pool_2)

    with tf.variable_scope('discriminator_lstm'):
        lstm_cell_fw = tf.contrib.rnn.BasicLSTMCell(
            num_units=32)
        lstm_cell_bw = tf.contrib.rnn.BasicLSTMCell(
            num_units=32)

        _, output_states = tf.nn.bidirectional_dynamic_rnn(
            cell_fw=lstm_cell_fw, cell_bw=lstm_cell_bw, inputs=max_pool_2, 
            dtype=tf.float32)
        print("fw_lstm_output: ", output_states[0].h)
        print("bw_lstm_output: ", output_states[1].h)

    lstm_output = tf.concat([output_states[0].h, output_states[1].h], 
                            axis=1)
    print("lstm_output: ", lstm_output)

    bnorm_3 = tf.layers.batch_normalization(lstm_output, name="batch_norm_3")
    print("batch_norm_3: ", bnorm_3)

    dense_1 = tf.layers.dense(
        inputs=lstm_output, units=NUM_CLASSES, 
        activation=tf.nn.relu, name="dense_1")
    print("dense_1.shape: ", dense_1.shape)

    softmax_output = tf.nn.softmax(dense_1, name="softmax")
    print("softmax_output.shape: ", softmax_output.shape)

    one_hot_label = tf.one_hot(
        indices=input_label - 1, depth=1, on_value=1, off_value=0,
        name="one_hot_label")
    one_hot_label = tf.reshape(one_hot_label, tf.shape(softmax_output))
    print("one_hot_label.shape: ", one_hot_label.shape)

    discriminator_loss = tf.losses.softmax_cross_entropy(
        one_hot_label, softmax_output)
    print("discriminator_loss: ", discriminator_loss)

    discriminator_loss_summary = tf.summary.scalar(
        "cross-entropy-loss-" + str(current_epoch), tf.convert_to_tensor(discriminator_loss))

    discriminator_optimizer = tf.train.AdamOptimizer()
    discriminator_train_operation = discriminator_optimizer.minimize(discriminator_loss)
    
    
    # Generator Network
    with tf.variable_scope('generator_lstm'):
        vanilla_lstm_cell_fw = tf.contrib.rnn.BasicLSTMCell(
            num_units=64)
        vanilla_lstm_cell_bw = tf.contrib.rnn.BasicLSTMCell(
            num_units=64)
    
        vanilla_rnn_outputs, _ = tf.nn.bidirectional_dynamic_rnn(
            cell_fw=vanilla_lstm_cell_fw, cell_bw=vanilla_lstm_cell_bw, 
            inputs=embedded_sequence, dtype=tf.float32, time_major=True)
        print("vanilla_lstm_outputs_fw: ", vanilla_rnn_outputs[0])
        print("vanilla_lstm_outputs_bw: ", vanilla_rnn_outputs[1])
        
    def perform_vocab_softmax(word_tensor):
        dense_word_1 = tf.layers.dense(
            inputs=word_tensor, units=VOCAB_SIZE, 
            activation=tf.nn.relu, name="dense_word_1")

#         word_hardmax = tf.contrib.seq2seq.hardmax(
#             tf.nn.softmax(dense_word_1), name="hardmax_word")
        
        return dense_word_1
        
    mapped_lstm_output = tf.map_fn(
        perform_vocab_softmax,
        vanilla_rnn_outputs[0],
        name='mapped_lstm'
    )
    print("mapped_lstm_output: ", mapped_lstm_output)
    
    generator_loss = tf.contrib.seq2seq.sequence_loss(
        logits=mapped_lstm_output,
        targets=input_sequence, 
        weights=tf.ones_like(
            input_sequence, dtype=tf.float32, name=None, optimize=True
        ),
        name='generator_loss'
    )
    print("generator_loss: ", generator_loss)
    
    generator_loss_summary = tf.summary.scalar(
        "generated-sequence-loss-" + str(current_epoch), tf.convert_to_tensor(generator_loss))
    
    generator_optimizer = tf.train.AdamOptimizer()
    generator_train_operation = generator_optimizer.minimize(generator_loss)

In [ ]:
with tf.Session(graph=graph_1) as sess:
    sess.run(tf.global_variables_initializer())
    
    writer = tf.summary.FileWriter(
        logdir="/home/v2john/tensorlogs/", graph=graph_1)
    
    epoch_reporting_interval = 10
    training_examples_fraction = 0.9
    training_examples_size = int(training_examples_fraction * len(labels))
    batch_size = 100
    training_epochs = 100
    num_batches = int(training_examples_size/batch_size)
    
    mini_epoch = 1
    for current_epoch in range(1, training_epochs + 1):
        for batch_number in range(num_batches):
            _, _, discriminator_loss, generator_loss = sess.run(
                [discriminator_train_operation, generator_train_operation, 
                 discriminator_loss_summary, generator_loss_summary], 
                feed_dict={
                    input_sequence: indexed_sequences[batch_number * batch_size : 
                                               (batch_number + 1) * batch_size],
                    input_label: labels[batch_number * batch_size : 
                                    (batch_number + 1) * batch_size]})
            writer.add_summary(discriminator_loss, mini_epoch)
            writer.add_summary(generator_loss, mini_epoch)
            writer.flush()
            mini_epoch += 1

        if (current_epoch % epoch_reporting_interval == 0):
            print("Training epoch: ", current_epoch, ", Loss: ", loss_var)
            
    training_predictions = sess.run(
        softmax_output, 
        feed_dict={
            input_sequence: indexed_sequences[:training_examples_size], 
            input_label: labels[:training_examples_size]
        })
    
    test_predictions = sess.run(
        softmax_output, 
        feed_dict={
            input_sequence: indexed_sequences[training_examples_size:], 
            input_label: labels[training_examples_size:]
        })
    
    writer.flush()
    writer.close()